In [ ]:
import docplex.mp
from docplex.mp.model import Model
import math

# input data about link travel time

In [ ]:
link = open("link1","r")
filelines = link.read().splitlines()
link = dict()
for i in range(1,len(filelines)):
    a = filelines[i]
    link[a[0:a.index(',')],a[a.index(',')+1:a.index(':')]]= int(a[a.index(':')+1:])
    link[a[a.index(',')+1:a.index(':')],a[0:a.index(',')]]= int(a[a.index(':')+1:])
link

# input data about route information

In [ ]:
route = open("route2","r")
filelines = route.read().splitlines()
path = dict()
for i in range(1,len(filelines)):
    a = filelines[i]
    path[a[0:a.index(':')]]= a[a.index(':')+1:]
path

calculate the turnover time for each bus route 

In [ ]:
path_cost = dict()
for i in path.keys():
    path_cost[i] = 2
for i in path.keys():
    a = path[i].split(',')
    b = 0
    while b <len(a)-1:
        path_cost[i] += link[a[b],a[b+1]]+2
        b+=1
for i in path.keys():
    path_cost[i] = path_cost[i]
path_cost

# input data about OD matrix

In [ ]:
OD = open("OD","r")
filelines = OD.read().splitlines()
odpair = dict()
for i in range(1,len(filelines)):
    a = filelines[i]
    odpair[a[0:a.index(',')],a[a.index(',')+1:a.index(':')]]=a[a.index(':')+1:]
odpair    

set the K value for each bus route

In [ ]:
K = dict()
for p in path.keys():
    a = 0
    for i in path[p].split(','):
        try:
            if int(i)>0:
                a+=1
                K[p,i]=a  
        except:
            continue
K

#  find the optimal passengers' travel paths based on transfer time

In [ ]:
travel = dict()
for od in odpair.keys():
    path_number = 0
    travel[od] = dict()
    O = []
    D = []
    for p in path.keys():
        if od[0] in path[p].split(','):
            O.append(p)
        if od[1] in path[p].split(','):
            D.append(p)
#     无换乘
    print('起点路线:',O,'终点路线:',D)
    a=list(set(O)&set(D))
    for b in a:
        if K[b,od[0]]<K[b,od[1]]:
            path_number += 1
            print('OD: %s,无换乘线路: %s,起点:%s,终点:%s' %(od,str(b),str(od[0]),str(od[1])))
            path_list = path[b].split(',')
            i = path[b].split(',').index(od[0])
            travel_list = [path_list[i]]
            cost = 0
            while i != path[b].split(',').index(od[1]):
                
                print(link[str(path_list[i]),str(path_list[i+1])])
                travel_list.append(path_list[i+1])
                cost += link[str(path_list[i]),str(path_list[i+1])]
                i += 1
            travel_list.append('成本:'+ str(cost))
            travel[od][path_number] = travel_list
            
    #     一次换乘    
    for o in O:
            for d in D:  
                if d != o:
                    commonstation = []
                    common_stations = list(set(path[o].split(','))&set(path[d].split(',')))
                    if len(common_stations) != 0:    
                        for a in common_stations:
                            if K[o,od[0]]<K[o,a] and K[d,od[1]]>K[d,a]:
                                if a != od[0] and a != od[1]:
                                    path_number += 1
                                    print('OD: %s,一次换乘线路: %s-%s,在站点%s换乘，起点:%s,终点:%s' %(od,o,d,a,str(od[0]),str(od[1])))
                                    path_list = path[o].split(',')
                                    i = path[o].split(',').index(od[0])
                                    travel_list = [path_list[i]]
                                    cost = 0
                                    while i != path[o].split(',').index(a):

                                        print(link[str(path_list[i]),str(path_list[i+1])])
                                        travel_list.append(path_list[i+1])
                                        cost += link[str(path_list[i]),str(path_list[i+1])]
                                        i += 1
                                    path_list = path[d].split(',')
                                    i = path_list.index(a)
                                    cost += 3
                                    while i != path[d].split(',').index(od[1]):

                                        print(link[str(path_list[i]),str(path_list[i+1])])
                                        travel_list.append(path_list[i+1])
                                        cost += link[str(path_list[i]),str(path_list[i+1])]
                                        i += 1    
                                    travel_list.append('成本:'+ str(cost))
                                    travel[od][path_number] = travel_list
        #     两次换乘
    for o in O:
        for d in D:
            for p in path.keys():
                cs_op = []
                cs_pd = []
                if p != o and p != d and o != d:
                    cs_op = list(set(path[o].split(','))&set(path[p].split(',')))
                    cs_pd = list(set(path[d].split(','))&set(path[p].split(',')))
                if len(cs_op)!= 0 and len(cs_pd)!= 0:
                    for i in cs_op:
                        for k in cs_pd:
                            if K[p,i]<K[p,k]:
                                if K[o,od[0]]<K[o,i] and K[d,k]<K[d,od[1]]:
                                    if i != od[0] and i != od[1] and k != od[0] and k != od[1]:
                                        path_number += 1
                                        print('OD: %s,两次换乘线路: %s-%s-%s,在站点%s和%s换乘，起点:%s,终点:%s' %(od,o,p,d,i,k,str(od[0]),str(od[1])))
                                        path_list = path[o].split(',')
                                        t = path[o].split(',').index(od[0])
                                        travel_list = [path_list[t]]
                                        cost = 0
                                        while t != path[o].split(',').index(i):

                                            print(link[str(path_list[t]),str(path_list[t+1])])
                                            travel_list.append(path_list[t+1])
                                            cost += link[str(path_list[t]),str(path_list[t+1])]
                                            t += 1
                                        path_list = path[p].split(',')
                                        t = path_list.index(i)
                                        cost += 3
                                        while t != path[p].split(',').index(k):

                                            print(link[str(path_list[t]),str(path_list[t+1])])
                                            travel_list.append(path_list[t+1])
                                            cost += link[str(path_list[t]),str(path_list[t+1])]
                                            t += 1    
                                        path_list = path[d].split(',')
                                        t = path_list.index(k)
                                        cost += 3
                                        while t != path[d].split(',').index(od[1]):

                                            print(link[str(path_list[t]),str(path_list[t+1])])
                                            travel_list.append(path_list[t+1])
                                            cost += link[str(path_list[t]),str(path_list[t+1])]
                                            t += 1    
                                        travel_list.append('成本:'+ str(cost))                                        
                                        travel[od][path_number] = travel_list
            #     三次换乘
    for o in O:
        for d in D:
            for p in path.keys():
                for q in path.keys():
                    cs_op = []
                    cs_pq = []
                    cs_qd = []
                    if p != o and p != d and q != o and q != d and q != p and o!=d:
                        cs_op = list(set(path[o].split(','))&set(path[p].split(',')))
                        cs_pq = list(set(path[p].split(','))&set(path[q].split(',')))
                        cs_qd = list(set(path[q].split(','))&set(path[d].split(',')))
                        if len(cs_op)!= 0 and len(cs_pq)!= 0 and len(cs_qd)!= 0:
                            for i in cs_op:
                                for k in cs_pq:
                                    for z in cs_qd:
                                        if i not in [od[0],od[1]] and k not in [od[0],od[1]] and z not in [od[0],od[1]]:
                                            if K[o,od[0]]<K[o,i] and K[p,i]<K[p,k] and K[q,k]<K[q,z] and K[d,z]<K[d,od[1]]: 
                                                if i != z:
                                                    path_number += 1
                                                    print('OD: %s,三次换乘线路: %s-%s-%s-%s,在站点%s和%s和%s换乘，起点:%s,终点:%s' %(str(od),str(o),str(p),str(q),str(d),str(i),str(k),str(z),str(od[0]),str(od[1])))       
                                                    path_list = path[o].split(',')
                                                    t = path[o].split(',').index(od[0])
                                                    travel_list = [path_list[t]]
                                                    cost = 0
                                                    while t != path[o].split(',').index(i):

                                                        print(link[str(path_list[t]),str(path_list[t+1])])
                                                        travel_list.append(path_list[t+1])
                                                        cost += link[str(path_list[t]),str(path_list[t+1])]
                                                        t += 1
                                                    path_list = path[p].split(',')
                                                    t = path_list.index(i)
                                                    cost += 3
                                                    while t != path[p].split(',').index(k):

                                                        print(link[str(path_list[t]),str(path_list[t+1])])
                                                        travel_list.append(path_list[t+1])
                                                        cost += link[str(path_list[t]),str(path_list[t+1])]
                                                        t += 1   
                                                    path_list = path[q].split(',')
                                                    t = path_list.index(k)
                                                    cost += 3
                                                    while t != path[q].split(',').index(z):

                                                        print(link[str(path_list[t]),str(path_list[t+1])])
                                                        travel_list.append(path_list[t+1])
                                                        cost += link[str(path_list[t]),str(path_list[t+1])]
                                                        t += 1
                                                    path_list = path[d].split(',')
                                                    t = path_list.index(z)
                                                    cost += 3
                                                    while t != path[d].split(',').index(od[1]):

                                                        print(link[str(path_list[t]),str(path_list[t+1])])
                                                        travel_list.append(path_list[t+1])
                                                        cost += link[str(path_list[t]),str(path_list[t+1])]
                                                        t += 1    
                                                    travel_list.append('成本:'+ str(cost))                                        
                                                    travel[od][path_number] = travel_list


calcualte the travel time for each passengers' path

In [ ]:
for od in odpair.keys():
    for i in range(1,len(travel[od])+1):
           travel[od][i][-1] = '成本:'+ str(int(travel[od][i][-1].split(':')[-1])+(len(travel[od][i])-1)*2)
    print(od,travel[od])

calculate the path flow for each passengers' path

In [ ]:
for od in odpair.keys():
    de = 0
    for i in range(1,len(travel[od])+1):
        a = int(travel[od][i][-1].split(':')[-1])
        de += math.exp(-1*a)
    for i in range(1,len(travel[od])+1):
        a = int(travel[od][i][-1].split(':')[-1])
        
        h = (int(odpair[od])*math.exp(-a))/de
        travel[od][i].append('流量:'+str(h))

calculate the total system travel time 

In [ ]:
ch = 0
for od in odpair.keys():   
    for i in range(1,len(travel[od])+1):  
        h = float(travel[od][i][-1].split(':')[-1])
        c = float(travel[od][i][-2].split(':')[-1])
        ch += c*h
print(ch)

calculate the flow on each bus link

In [ ]:
link_flow = dict()
for l in link.keys():
     link_flow[l] = 0
for od in odpair.keys():
    for i in range(1,len(travel[od])+1):
        a = travel[od][i][0:-2]
        b = 0
        h = float(travel[od][i][-1].split(':')[-1])
        while b < len(a)-1:
            link_flow[a[b],a[b+1]] += h
            b += 1
for l in link.keys():
     if link_flow[l] == 0:
        link_flow.pop(l)
link_flow        

set bus routes

In [ ]:
a = list(path.keys())
for i in a:
    if i not in ['1', '2','3', '4']:
        path.pop(i)
path

the bus link belongs to what bus routes 

In [ ]:
route_frequency = dict()
for l in link.keys():
    route_frequency[l] = []
for i in path.keys():
    a= path[i].split(',')
    b= 0
    c = 1
    while b < len(a)-1:
        route_frequency[a[b],a[b+1]].append(i)
        b += 1
for l in link.keys():
     if len(route_frequency[l])==0:
        route_frequency.pop(l)
route_frequency

calculate the flow on each bus link

In [ ]:
a = list(link_flow.keys())
for i in a:
    if i not in route_frequency.keys():
        link_flow.pop(i)
link_flow

Cplex package for calculate the linear pragramming problem

In [ ]:
m = Model(name='route')
f = m.continuous_var_dict(path.keys(),name='f')
d = m.continuous_var_dict(link_flow.keys(),name='d')
for i in f.keys():
    m.add_constraint(f[i]>=0)
for i in d.keys():
    m.add_constraint(d[i]>=0)
for i in link_flow.keys():
    a = m.sum(f[j] for j in route_frequency[i])
    m.add_constraint(link_flow[i]-d[i]-50*a <=0 )

a = m.sum(path_cost[j]*f[j] for j in path.keys())
m.add_constraint(f['1']==f['2'])
m.add_constraint(f['3']==f['4'])
m.add_constraint(a<=70)
a = m.sum(d[i] for i in d.keys())
m.minimize(20 * a+ch)
m.print_information()   

In [ ]:
s = m.solve()
m.print_solution(print_zeros=True)

In [ ]:
(s.objective_value-ch)

In [ ]:
ch

In [ ]:
# for od in odpair.keys():
#     O = []
#     D = []
#     for p in path.keys():
#         if od[0] in path[p].split(','):
#             O.append(p)
#         if od[1] in path[p].split(','):
#             D.append(p)
# #     无换乘
#     print('起点路线:',O,'终点路线:',D)
#     a=list(set(O)&set(D))
#     for b in a:
#         if K[b,od[0]]<K[b,od[1]]:
#             print('OD: %s,无换乘线路: %s,起点:%s,终点:%s' %(od,str(b),str(od[0]),str(od[1])))
#             path
            
# #     一次换乘    
#     for o in O:
#             for d in D:  
#                 if d != o:
#                     commonstation = []
#                     common_stations = list(set(path[o].split(','))&set(path[d].split(',')))
#                     if len(common_stations) != 0:    
#                         for a in common_stations:
#                             if K[o,od[0]]<K[o,a] and K[d,od[1]]>K[d,a]:
#                                 if a != od[0] and a != od[1]:
#                                     print('OD: %s,一次换乘线路: %s-%s,在站点%s换乘，起点:%s,终点:%s' %(od,o,d,a,str(od[0]),str(od[1])))
# #     两次换乘
#     for o in O:
#         for d in D:
#             for p in path.keys():
#                 cs_op = []
#                 cs_pd = []
#                 if p != o and p != d and o != d:
#                     cs_op = list(set(path[o].split(','))&set(path[p].split(',')))
#                     cs_pd = list(set(path[d].split(','))&set(path[p].split(',')))
#                 if len(cs_op)!= 0 and len(cs_pd)!= 0:
#                     for i in cs_op:
#                         for k in cs_pd:
#                             if K[p,i]<K[p,k]:
#                                 if K[o,od[0]]<K[o,i] and K[d,k]<K[d,od[1]]:
#                                     if i != od[0] and i != od[1] and k != od[0] and k != od[1]:
#                                         print('OD: %s,两次换乘线路: %s-%s-%s,在站点%s和%s换乘，起点:%s,终点:%s' %(od,o,p,d,i,k,str(od[0]),str(od[1])))
#     #     三次换乘
#     for o in O:
#         for d in D:
#             for p in path.keys():
#                 for q in path.keys():
#                     cs_op = []
#                     cs_pq = []
#                     cs_qd = []
#                     if p != o and p != d and q != o and q != d and q != p and o!=d:
#                         cs_op = list(set(path[o].split(','))&set(path[p].split(',')))
#                         cs_pq = list(set(path[p].split(','))&set(path[q].split(',')))
#                         cs_qd = list(set(path[q].split(','))&set(path[d].split(',')))
#                         if len(cs_op)!= 0 and len(cs_pq)!= 0 and len(cs_qd)!= 0:
#                             for i in cs_op:
#                                 for k in cs_pq:
#                                     for z in cs_qd:
#                                         if i not in [od[0],od[1]] and k not in [od[0],od[1]] and z not in [od[0],od[1]]:
#                                             if K[o,od[0]]<K[o,i] and K[p,i]<K[p,k] and K[q,k]<K[q,z] and K[d,z]<K[d,od[1]]: 
#                                                 if i != z:
#                                                     print('OD: %s,三次换乘线路: %s-%s-%s-%s,在站点%s和%s和%s换乘，起点:%s,终点:%s' %(str(od),str(o),str(p),str(q),str(d),str(i),str(k),str(z),str(od[0]),str(od[1])))                                                       